In [5]:
import torch  
  
def householder_transformation(v, target):  
    """  
    生成一个Householder变换矩阵,将v旋转到target。  
    """  
    # 确保v和target是单位向量  
    v = v / torch.norm(v)  
    target = target / torch.norm(target)  
      
    # 计算差向量w并归一化得到v  
    w = v - target  
    u = w / torch.norm(w)  
      
    # 构造Householder矩阵H  
    I = torch.eye(len(v))  
    H = I - 2.0 * torch.outer(u, u)  
      
    return H  
  
# 示例  
v = torch.tensor([0.55, 0.66, 0.77]) 
print('原始向量:', v.numpy())
target = torch.tensor([0.0, 1.0, 0.0])  
H = householder_transformation(v, target)  
  
# 测试变换  
v_rotated = torch.matmul(H, v)  
print('旋转矩阵:', H.numpy())
print("旋转后的向量:", v_rotated.numpy())  


原始向量: [0.55 0.66 0.77]
旋转矩阵: [[ 0.4688927   0.47673133 -0.74355024]
 [ 0.47673133  0.5720775   0.66742384]
 [-0.74355024  0.66742384 -0.04097033]]
旋转后的向量: [ 5.9604645e-08  1.1536897e+00 -5.2154064e-08]


In [ ]:
import numpy as np  
from scipy.optimize import minimize 
from scipy.spatial.transform import Rotation as R  
  
def rotation_matrix_to_euler_angles(matrix):  
    """  
    通过优化过程从旋转矩阵中估计欧拉角。  
    """  
    def loss_function(angles):  
        # 使用给定的欧拉角构造旋转矩阵  
        r = R.from_euler('zyz', angles, degrees=False)  
        rotation_estimated = r.as_matrix()  
        # 计算估计的旋转矩阵与实际矩阵之间的误差  
        loss = np.linalg.norm(rotation_estimated - matrix)  
        return loss  
  
    # 使用优化方法寻找最小化误差的欧拉角  
    initial_guess = [0, 0, 0]  
    result = minimize(loss_function, initial_guess, method='Nelder-Mead')  
  
    if result.success:  
        return result.x  # 返回估计的欧拉角  
    else:  
        raise ValueError("Optimization failed to converge")  
  
# 示例：给定一个2阶球谐旋转矩阵（这里仅为示意，你需要替换成实际的旋转矩阵）  
# matrix_example = R.from_euler('zyz', [0.5, -0.3, 0.7], degrees=False).as_matrix()  
  
# 从旋转矩阵中估计欧拉角  
estimated_angles = rotation_matrix_to_euler_angles(v_rotated.numpy())  
print("Estimated Euler Angles (zyz):", estimated_angles)  


Estimated Euler Angles (zyz): [ 1.89310249  0.6544116  -2.75459576]


In [13]:
import e3nn.o3 as o3

ground_angles = o3.matrix_to_angles(H)
R = torch.Tensor(H)
torch.allclose(torch.det(R), R.new_tensor(1))
torch.det(R)


# print("Ground angles (zyz):", ground_angles)

AssertionError: 

In [6]:
import sys
import os
import pubchempy as pcp
import random
import numpy as np
from pyscf import gto

# sys.path.append(os.path.dirname(os.path.dirname(__file__)))


# Function to fetch molecules from PubChem
def fetch_pubchem_molecules(num_molecules=3):
    # You can modify this to select molecules with different sizes
    molecules = []
    cid_list = pcp.get_cids('organic compounds', 'name')  # Get list of CIDs from PubChem

    for cid in cid_list[:num_molecules]:
        compound = pcp.Compound.from_cid(cid)
        # Get 3D structure of the compound
        mol = compound.to_dict(properties=['atoms', '3d'])
        molecules.append(mol)
    
    return molecules

# Save molecule data in XYZ format
def save_xyz(molecule, output_dir):
    atom_pos = molecule['3d']
    atom_num = [atom['element'] for atom in molecule['atoms']]
    
    # Generate XYZ string
    atoms = [f"{num} {pos[0]} {pos[1]} {pos[2]}" for num, pos in zip(atom_num, atom_pos)]
    atoms = '\n'.join(atoms)

    # Create a valid filename using CID
    mol_name = str(molecule['cid'])
    xyz_filename = os.path.join(output_dir, f"{mol_name}.xyz")
    
    with open(xyz_filename, 'w') as f:
        f.write(f"{len(atom_pos)}\n")
        f.write(f"Generated from PubChem: CID {mol_name}\n")
        f.write(atoms)

    print(f"Saved XYZ for molecule {mol_name} at {xyz_filename}")

if __name__ == "__main__":
    output_dir = "/data/pubchem_xyz"
    # Create output directory if not exists
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Fetch 329 molecules from PubChem
    molecules = fetch_pubchem_molecules(num_molecules=3)
    
    # Loop over each molecule and save their XYZ data
    for molecule in molecules:
        save_xyz(molecule, output_dir)


In [12]:


# 查询特定化学物质的CID，例如 "aspirin"
pcp.get_all_sources()



['001Chemical',
 '10X CHEM',
 '1st Scientific',
 '3A SpeedChemical Inc',
 '3B Scientific (Wuhan) Corp',
 '3WAY PHARM INC',
 '4C Pharma Scientific Inc',
 'A&J Pharmtech CO., LTD.',
 'A1 BioChem Labs',
 'A2B Chem',
 'A2Z Chemical',
 'AA BLOCKS',
 'AAA Chemistry',
 'Aaron Chemicals LLC',
 'AAT Bioquest',
 'AbaChemScene',
 'Abacipharm Corp',
 'ABBLIS Chemicals',
 'Abbott Labs',
 'abcr GmbH',
 'Abe Lab, University of Texas MD Anderson Cancer Center',
 'ABI Chem',
 'AbMole Bioscience',
 'AbovChem LLC',
 'Abu Montakim Tareq, International Islamic University Chittagong',
 'Acadechem',
 'Accela ChemBio Inc.',
 'Ace Therapeutics',
 'Acemol',
 'Aceschem Inc',
 'Acesobio',
 'Achem-Block',
 'Achemica',
 'Achemo Scientific Limited',
 'Achemtek',
 'Acmec Biochemical',
 'ACO Pharm Screening Compound',
 'Acorn PharmaTech Product List',
 'ACT Chemical',
 'Activate Scientific',
 'Active Biopharma',
 'Adooq BioScience',
 'Advanced Technology & Industrial Co., Ltd.',
 'AEchem Scientific Corp., USA',
 'Agio

In [ ]:
mols = torch.load("/data/pubchem/data.0000.lmdb")
output_dir = "/data/pubchem_xyz/"
mol_idxes = list(range(len(mols)))
random.shuffle(mol_idxes)
print(f"first 1000 element idx of mol_idxes: {mol_idxes[:1000]}")
for i in mol_idxes:
    #get the name of xyz file
    molecule = mols[i]
    mol_name = os.path.join(output_dir,str(molecule['cid']))
    print(f"Running {molecule['cid']}, i {i}...")
    if os.path.exists(os.path.join(output_dir,mol_name + '_grad.npy')):
        print(f"Skip {mol_name}...")
        continue
    charge = molecule['data']['pubchem']['charge']
    multiplicity = molecule['data']['pubchem']['multiplicity']
    print('charge:', charge, 'multiplicity:', multiplicity)
    atom_pos = molecule['data']['pubchem']['B3LYP@PM6']['atoms']['coords']['3d']
    atom_num = molecule['data']['pubchem']['B3LYP@PM6']['atoms']['elements']['number']
    atom_pos = np.array(atom_pos).reshape(-1, 3)
    np.savez(os.path.join(output_dir, str(molecule['cid']) + '_pos.npy'), atom_pos=atom_pos, atom_num=atom_num)
    # convert to xyz string
    # num2atom = {1: 'H', 6: 'C', 7: 'N', 8: 'O', 9: 'F'}
    atom = [' '.join([str(atom_num[i]), str(atom_pos[i][0]), str(atom_pos[i][1]), str(atom_pos[i][2])]) for i in range(len(atom_num))]
    atom = '\n'.join(atom)
    with open(os.path.join(output_dir, str(molecule['cid']) + '.xyz'), 'w') as f:
        f.write(f"{len(atom_num)}\n")
        f.write(f"Generated from PubChem: CID {molecule['cid']}\n")
        f.write(atom)